In [ ]:
import json
from typing import List, Any
from os import listdir, system
from os.path import isfile, join
import numpy as np
import re
import copy
from utils import *
from mincov import *

In [ ]:
PATH_MINCOV = "./salida2/PetrinetSE-original"

In [ ]:
# Opening JSON file
f = open('matriz.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)
matriz_incidencia = data["Incidencia"]
matriz_incidencia_transpuesta = np.transpose(matriz_incidencia)
marcado_inicial = data["Marcado"]

N_PLAZAS = len(matriz_incidencia)
N_TRANSICIONES = len(matriz_incidencia[0])

In [ ]:
# Buscamos las plazas con no más de una entrada o salida
plazas_simples,\
plazas_complejas,\
plazas_dos_entradas_una_salida,\
plazas_dos_salidas_una_entrada,\
plazas_complejas_total = clasificar_plazas(matriz_incidencia)

# Buscamos las transiciones a evitar y tambien guardamos unos keymap de:
# *) Transicion relacionada con la salida  con plazas_dos_entradas_una_salida
# *) Transicion relacionada con la entrada con plazas_dos_salidas_una_entrada

transiciones_indeseadas, \
transiciones_indeseadas_totales, \
transiciones_plazas_dos_entradas_una_salida, \
transiciones_plazas_dos_salidas_una_entrada, \
dict_plazas_dos_entradas_una_salida, \
dict_plazas_dos_salidas_una_entrada = clasificar_transiciones(matriz_incidencia, plazas_complejas, plazas_dos_entradas_una_salida, plazas_dos_salidas_una_entrada)



# Busco los trenes comprendidos por:
#  *) Caminos simples (plazas con 1 entrada y 1 salida)
#  *) Recursos internos
#  *) plazas de 1 in y 2 out y 2 in y 1 out
transiciones_recorridas = []
caminos_simples_encontrados = []
caminos_con_inicio_fin_complejo_encontrados = []
transiciones_de_caminos_simples_encontrados = []
transiciones_de_caminos_con_inicio_fin_complejo_encontrados = []
transiciones_borde = transiciones_indeseadas_totales.copy()
while (len(plazas_simples) > 0): # Voy a intentar recorrer todas las plazas de 1 entrda y 1 salida
    res = [plazas_simples[0]] # Guardo la primera plaza
    res_transiciones_usadas = set()
    res_con_plazas_especiales = [plazas_simples[0]] # plazas especiales son aquellas con 2 entradas y 1 salida mazimo o viceversa 
    res_transiciones_usadas_con_plazas_especiales = set()
    for plaza in res: #recorro todas las plazas de res para buscar plazas conectadas a esta y agregarlas al mismo
        plaza_index = plazas_simples.index(plaza)
        plazas_simples.pop(plaza_index)
        for t in range(N_TRANSICIONES):
            if (matriz_incidencia[plaza - 1][t] != 0): # Si es distinto de 0 significa que esta T esta conectada a la plaza que estoy analizando
                res_transiciones_usadas_con_plazas_especiales.add(t+1)
                res_transiciones_usadas.add(t+1)
                if (not t+1 in transiciones_recorridas): # Solo me interesa si no la recorri
                    if (not t+1 in transiciones_indeseadas): # La transicion no sea de un recurso compartido
                        transiciones_recorridas.append(t+1)
                        if(not t+1 in transiciones_indeseadas_totales): # Si la transicion tiene max 1 entrada/salida o es de un recurso interno busco las plazas atadas a esta
                            for p in range(N_PLAZAS):
                                if (matriz_incidencia[p][t] != 0 and p+1 in plazas_simples and not p+1 in res): # Si la plaza esta conectado a esta transicion, es simple (1 in/out max) y no esta en res
                                    res.append(p + 1)
                                    res_con_plazas_especiales.append(p+1)
                        # Significa que la transicion esta conectada a una plaza de 2i1o o 2o1i
                        else:
                            try_add_to_train_2i1o_1i2o(
    t,
    res_con_plazas_especiales,
    res_transiciones_usadas_con_plazas_especiales,
    transiciones_plazas_dos_entradas_una_salida,
    dict_plazas_dos_entradas_una_salida,
    plazas_dos_entradas_una_salida,
    transiciones_plazas_dos_salidas_una_entrada,
    dict_plazas_dos_salidas_una_entrada,
    plazas_dos_salidas_una_entrada,
    transiciones_borde,
    matriz_incidencia)
                    else: # No recurrida pero conectada a un recurso compartido
                        for p in range(N_PLAZAS):
                                # Si la plaza esta conectado a esta transicion, es simple (1 in/out max) y no esta en res
                                if (matriz_incidencia[p][t] != 0 and p+1 in plazas_simples and not p+1 in res): 
                                    res.append(p + 1)
                                    res_con_plazas_especiales.append(p+1)
                                    transiciones_borde.add(t+1)

    caminos_simples_encontrados.append(res)
    caminos_con_inicio_fin_complejo_encontrados.append(res_con_plazas_especiales)
    transiciones_de_caminos_simples_encontrados.append(res_transiciones_usadas)
    transiciones_de_caminos_con_inicio_fin_complejo_encontrados.append(res_transiciones_usadas_con_plazas_especiales)


# Agrega recursos compartidos
for p in plazas_complejas:
    encontro = False
    transiciones_conectadas = set()
    for t in range(N_TRANSICIONES):
        if matriz_incidencia[p-1][t] != 0:
            transiciones_conectadas.add(t+1) # Guarda transiciones conectadas al recurso compartido
    for t_conjunto in range(len(transiciones_de_caminos_con_inicio_fin_complejo_encontrados)):
        cantidad_encontradas = 0
        temp = []
        temp2 = []
        for tplaza in transiciones_conectadas:
            if tplaza in transiciones_de_caminos_con_inicio_fin_complejo_encontrados[t_conjunto]:
                cantidad_encontradas+=1
                temp2.append(tplaza)
            else:
                temp.append(tplaza)
        if cantidad_encontradas > 1:
            encontro = True
            # print("Deberia agregar la plaza", p, "en el arreglo", caminos_con_inicio_fin_complejo_encontrados[t_conjunto], "y", temp, "en ", transiciones_de_caminos_con_inicio_fin_complejo_encontrados[t_conjunto])
            caminos_con_inicio_fin_complejo_encontrados[t_conjunto].append(p)
            for t_faltantes in temp:
                transiciones_de_caminos_con_inicio_fin_complejo_encontrados[t_conjunto].add(t_faltantes)
            for t_no_faltantes in temp2:
                transiciones_borde.remove(t_no_faltantes)
                for p_conectados in range(N_PLAZAS):
                    if(matriz_incidencia[p_conectados-1][t_no_faltantes-1] != 0 and not p_conectados+1 in caminos_con_inicio_fin_complejo_encontrados[t_conjunto]):
                        transiciones_borde.add(t_no_faltantes) # Si hay alguna plaza que no forme parte del subconujunto agrego la transicion como borde
            # print("Deberia agregar la plaza", p, "en el arreglo", caminos_con_inicio_fin_complejo_encontrados[t_conjunto], "y", temp, "en ", transiciones_de_caminos_con_inicio_fin_complejo_encontrados[t_conjunto])
            break
    if not encontro:
        caminos_con_inicio_fin_complejo_encontrados.append([p])
        transiciones_de_caminos_con_inicio_fin_complejo_encontrados.append(transiciones_conectadas)

            
# Calculo Matriz Relacion:
matriz_relacion = [] # 1 columna por transicion borde y 1 fila por subred
transiciones_con_plazas_aux = []
transiciones_borde = list(transiciones_borde)
for i, conjunto in enumerate(transiciones_de_caminos_con_inicio_fin_complejo_encontrados):
    temp = []
    temp_transiciones_con_plazas_aux = []
    for t in transiciones_borde:
        if t in conjunto:
            temp.append(1)
            for p in range(N_PLAZAS):
                if matriz_incidencia[p][t-1] < 0 and not p+1 in caminos_con_inicio_fin_complejo_encontrados[i]:
                    temp_transiciones_con_plazas_aux.append(t)
                    break
        else:
            temp.append(0)
    matriz_relacion.append(temp)
    transiciones_con_plazas_aux.append(temp_transiciones_con_plazas_aux)


# Obtengo la matriz incidencias
matriz_incidencia_caminos_complejos = []
for numero_camino in range(len(caminos_simples_encontrados)):
    temp_padre = []
    for indice_p in range(len(caminos_con_inicio_fin_complejo_encontrados[numero_camino])):
        temp = []
        for t in transiciones_de_caminos_con_inicio_fin_complejo_encontrados[numero_camino]:
           temp.append(matriz_incidencia[caminos_con_inicio_fin_complejo_encontrados[numero_camino][indice_p] - 1][t-1])
        temp_padre.append(temp)
    for tt in transiciones_con_plazas_aux[numero_camino]:
        temp = []
        for t in transiciones_de_caminos_con_inicio_fin_complejo_encontrados[numero_camino]:
            if t==tt:
                temp.append(-1)
            else:
                temp.append(0)
        temp_padre.append(temp)
        caminos_con_inicio_fin_complejo_encontrados[numero_camino].append(tt*-1)
    matriz_incidencia_caminos_complejos.append(temp_padre)
    # print("")

# TODO: Ver si en un futuro se puede hacer que transiciones_de_caminos_con_inicio_fin_complejo_encontrados contenga lista)
for i, transiciones in enumerate(transiciones_de_caminos_con_inicio_fin_complejo_encontrados):
    transiciones_de_caminos_con_inicio_fin_complejo_encontrados[i] = list(transiciones)

print("Marcado Inicial:",marcado_inicial)
print("\nCamino con matriz_relacionentradas y salidas:")
print("Plazas:",caminos_con_inicio_fin_complejo_encontrados)
print("Transiciones:", transiciones_de_caminos_con_inicio_fin_complejo_encontrados)
print("M Incidencia:",matriz_incidencia_caminos_complejos)
# print("Tamaño matriz relacion", len(transiciones_borde), "x", len(caminos_con_inicio_fin_complejo_encontrados))
print("Transiciones borde", transiciones_borde)
print("Matriz relacion", matriz_relacion)
print("Plazas aux", transiciones_con_plazas_aux)

# Closing file
f.close()
print("\n--------------SOY UN SEPARADOR--------------\n")



"""
matriz_incidencia_... son entradas del algoritmo de gabi
- Ejecutar algoritmo de gabi con cada una de esas entradas
- Renombrar salidas del algoritmo de gabi con el patrón mincov_out_...etc
"""

mincov_inputs = []
for i, matriz in enumerate(matriz_incidencia_caminos_complejos):
    mincov_inputs.append(generate_mincov_json_input(i,matriz, set(), caminos_con_inicio_fin_complejo_encontrados, marcado_inicial))
    for plazas_temp_con_mark in combinations(transiciones_con_plazas_aux[i]):
        if len(plazas_temp_con_mark) > 0:
            set_to_string(plazas_temp_con_mark)
            mincov_input_filename = generate_mincov_json_input(i,matriz, plazas_temp_con_mark, caminos_con_inicio_fin_complejo_encontrados, marcado_inicial)
            mincov_inputs.append(mincov_input_filename)
mincov_inputs.append(generate_mincov_json_input_general(matriz_incidencia, marcado_inicial))



for mincov_input in mincov_inputs:
    system(f"{PATH_MINCOV} -a {mincov_input} > /dev/null")
    sufix = re.search(r".*incidencia_(.*)", mincov_input).groups()[0]
    system(f"mv ./mincov_out.json ./salida/mincov_out_{sufix}.json ")







"""
Quiero que el programa me guarde una lista por cada subred que a
su vez tenga una lista por cada combinacion de salida (las 
combinaciones se dan si tiene plaza auxiliares ya que estas pueden
estar habilitadas o deshabilitadas). Dentro de esta ultima lista 
necesito todos los estados de la salidas y sus respectivas relaciones
"""

PREFIX = "mincov_out_"
salidasMinCov = [f for f in sorted(listdir("salida")) if f.find(PREFIX) != -1]
lista_arboles_de_alcanzabilidad = []

index_nodos = 0
for i, subred in enumerate(caminos_con_inicio_fin_complejo_encontrados):
    salidas_de_la_subred = [s for s in salidasMinCov if s.startswith(f"{PREFIX}{i}")]
    if len(salidas_de_la_subred) > 0:
        lista_arboles_de_alcanzabilidad.append({})
        arboles = lista_arboles_de_alcanzabilidad[i]

        for s in salidas_de_la_subred:
            plazas_aux_con_marcadoinicial = s[s.index(str(i))+2:].split(".")[0]
            identificador_posibilidad = plazas_aux_con_marcadoinicial if plazas_aux_con_marcadoinicial != "" else "none"
            arboles[identificador_posibilidad], index_nodos  = getArbolFromSalida(
                s,
                i,
                index_nodos,
                identificador_posibilidad=="none",
                transiciones_de_caminos_con_inicio_fin_complejo_encontrados
                )

# print("aca empieza")
print(lista_arboles_de_alcanzabilidad)
print("\n--------------SOY UN SEPARADOR--------------\n")
                            

# # # Recoro todas las subredes y le envio la lista de nodos, lista de plazas con su nombre global incluyendo las auxiliares y marcado inicial global
# for indice, subred in enumerate(lista_arboles_de_alcanzabilidad):
#     completar_nodo(subred["none"]["nodos"], caminos_con_inicio_fin_complejo_encontrados[indice], marcado_inicial)
#     subred["none"]["completo"] = True
# # print(lista_arboles_de_alcanzabilidad[0]["none"]["nodos"])

interelacion_subredes = []

# for indice, subred in enumerate(lista_arboles_de_alcanzabilidad):
#     completarSubred(indice, subred)

completar_subred(0, lista_arboles_de_alcanzabilidad[0], matriz_incidencia_transpuesta, matriz_relacion, lista_arboles_de_alcanzabilidad, transiciones_borde, caminos_con_inicio_fin_complejo_encontrados, marcado_inicial, N_PLAZAS)

for i, arboles in enumerate(lista_arboles_de_alcanzabilidad):
    # print(arboles)
    print("a", arboles)
    generate_mincov_json_filled(arboles, i)


print(json.dumps(lista_arboles_de_alcanzabilidad))

arbol_completo = join_tree(lista_arboles_de_alcanzabilidad)
print(arbol_completo)
generate_mincov_json_filled2(arbol_completo)

print("------------------ FINAL ------------------")
print(lista_arboles_de_alcanzabilidad)
print(arbol_completo)